In [1]:
import pandas as pd

In [2]:
# "In NCANDS, a victim is defined as a child for whom the State determined at least one 
#  maltreatment was substantiated or indicated"

victims_df = pd.read_csv("Data/victims_2011.txt", sep='\t')
total_foster_df = pd.read_csv("Data/total_foster.txt", sep='\t')
in_care_df = pd.read_csv("Data/in_care.txt", sep='\t')
adopted_df = pd.read_csv("Data/adopted.txt", sep='\t')
tpr_df = pd.read_csv("Data/tpr.txt", sep='\t')
victims_by_age_df = pd.read_csv("Data/victims_by_age.txt", sep='\t')
perpetrators_df = pd.read_csv("Data/perpetrators.txt", sep='\t')

FileNotFoundError: [Errno 2] File victims_2011.txt does not exist: 'victims_2011.txt'

In [21]:
adopted_df.head()

,Status,State,FY 2010,FY 2011,FY 2012,FY 2013,FY 2014,FY 2015,FY 2016,FY 2017,FY 2018,FY 2019
0,Adopted,Alabama,606.0,447.0,587.0,532.0,544.0,512.0,503.0,504.0,712.0,738.0
1,Adopted,Alaska,336.0,293.0,309.0,328.0,367.0,357.0,312.0,366.0,353.0,396.0
2,Adopted,Arizona,2045.0,2275.0,2275.0,2522.0,3090.0,3236.0,3654.0,4298.0,3926.0,3373.0
3,Adopted,Arkansas,597.0,591.0,703.0,691.0,743.0,740.0,758.0,971.0,1001.0,984.0
4,Adopted,California,6459.0,5710.0,5938.0,5541.0,5471.0,6079.0,6542.0,6524.0,6966.0,6981.0


In [4]:
victims_df = victims_df.rename(columns = {"Number":"Victim Population", "Rate per 1,000":"Victim Rate per 1,000"})
perpetrators_df = perpetrators_df.rename(columns = {"Unique count Perpetrators":"Perpetrators"})

In [ ]:
# IDEA: 
# 1. Join Raw Foster with other tables by state (with  measures such as victims, fatalities, perpetrators...)
# 2. Create new column in joined table that describes population in care as either decreasing or increasing 
# (FY2019-FY2010 is negative or positive) 
# 3. Get rid of state column (may join with predicted and actual dataframe later)
# 4. Make a model to predict using the alternate measures whether the state's population in care is 
# increasing or decreasing

In [22]:
in_care = in_care_df.copy()
tpr = tpr_df.copy()
adopted = adopted_df.copy()

In [23]:
adopted["Difference"] = adopted["FY 2019"] - adopted["FY 2010"]
adopted.head()

,Status,State,FY 2010,FY 2011,FY 2012,FY 2013,FY 2014,FY 2015,FY 2016,FY 2017,FY 2018,FY 2019,Difference
0,Adopted,Alabama,606.0,447.0,587.0,532.0,544.0,512.0,503.0,504.0,712.0,738.0,132.0
1,Adopted,Alaska,336.0,293.0,309.0,328.0,367.0,357.0,312.0,366.0,353.0,396.0,60.0
2,Adopted,Arizona,2045.0,2275.0,2275.0,2522.0,3090.0,3236.0,3654.0,4298.0,3926.0,3373.0,1328.0
3,Adopted,Arkansas,597.0,591.0,703.0,691.0,743.0,740.0,758.0,971.0,1001.0,984.0,387.0
4,Adopted,California,6459.0,5710.0,5938.0,5541.0,5471.0,6079.0,6542.0,6524.0,6966.0,6981.0,522.0


In [6]:
tpr["Difference"] = tpr["FY 2019"] - tpr["FY 2010"]
tpr.head()

,Status,State,FY 2010,FY 2011,FY 2012,FY 2013,FY 2014,FY 2015,FY 2016,FY 2017,FY 2018,FY 2019,Difference
0,TPR,Alabama,757.0,701.0,543.0,615.0,573.0,461.0,421.0,533.0,514.0,523.0,-234.0
1,TPR,Alaska,381.0,389.0,387.0,399.0,366.0,356.0,425.0,432.0,430.0,375.0,-6.0
2,TPR,Arizona,2174.0,2220.0,2299.0,2692.0,2841.0,3006.0,3379.0,3295.0,2966.0,2687.0,513.0
3,TPR,Arkansas,812.0,742.0,543.0,642.0,757.0,757.0,795.0,795.0,809.0,787.0,-25.0
4,TPR,California,7230.0,6928.0,6021.0,6562.0,6379.0,6617.0,6712.0,7107.0,7098.0,7262.0,32.0


In [7]:
in_care["Difference"] = in_care["FY 2019"] - in_care["FY 2010"]
in_care.head()

,State,FY 2010,FY 2011,FY 2012,FY 2013,FY 2014,FY 2015,FY 2016,FY 2017,FY 2018,FY 2019,Difference
0,Alabama,5350.0,5253.0,4561.0,4435.0,4526.0,4745.0,5053.0,5631.0,5930.0,5783.0,433.0
1,Alaska,1791.0,1835.0,1854.0,1982.0,2190.0,2653.0,2824.0,2739.0,2781.0,2879.0,1088.0
2,Arizona,9930.0,10883.0,13461.0,14399.0,16246.0,17738.0,17118.0,15031.0,13360.0,13330.0,3400.0
3,Arkansas,3756.0,3732.0,3711.0,3797.0,3806.0,4548.0,4880.0,4776.0,4234.0,4127.0,371.0
4,California,56202.0,54754.0,54553.0,55383.0,56771.0,55983.0,54685.0,51869.0,52337.0,51268.0,-4934.0


In [8]:
status = []
for index, row in in_care.iterrows():
    difference = row["Difference"]
    if difference > 0:
        status.append("Increasing")
    else:
        status.append("Decreasing")
        
in_care["Status"] = status

In [9]:
in_care = in_care[["State","Status"]]

In [19]:
tpr = tpr.rename(columns = {"Difference":"TPR Difference"})
tpr = tpr[["State","TPR Difference"]]

In [29]:
adopted = adopted.rename(columns = {"Difference":"Adopted Difference"})
adopted = adopted[["State","Adopted Difference"]]

In [12]:
victims_df["State"] = victims_df["State"].astype(str)
victims_df["State"] = victims_df["State"].str.rstrip()
victims_df = victims_df[["State","Victim Population","Victim Rate per 1,000"]]

In [13]:
perpetrators_df["State"] = perpetrators_df["State"].astype(str)
perpetrators_df["State"] = perpetrators_df["State"].str.rstrip()

In [14]:
in_care_victims = victims_df.merge(in_care, how="inner", on = "State")
in_care_victims.head()

,State,Victim Population,"Victim Rate per 1,000",Status
0,Alabama,8601.0,7.63,Increasing
1,Alaska,2898.0,15.38,Increasing
2,Arizona,8708.0,5.36,Increasing
3,Arkansas,11105.0,15.63,Increasing
4,California,80100.0,8.64,Decreasing


In [15]:
in_care_victims_perps = perpetrators_df.merge(in_care_victims, how = "inner", on ="State")
in_care_victims_perps.head()

,State,Unique count Perpetrators,Victim Population,"Victim Rate per 1,000",Status
0,Alabama,7260.0,8601.0,7.63,Increasing
1,Alaska,2238.0,2898.0,15.38,Increasing
2,Arizona,9405.0,8708.0,5.36,Increasing
3,Arkansas,9552.0,11105.0,15.63,Increasing
4,California,62574.0,80100.0,8.64,Decreasing


In [20]:
in_care_victims_perps_tpr = tpr.merge(in_care_victims_perps, how = "inner", on ="State")
in_care_victims_perps_tpr.head()

,State,TPR Difference,Unique count Perpetrators,Victim Population,"Victim Rate per 1,000",Status
0,Alabama,-234.0,7260.0,8601.0,7.63,Increasing
1,Alaska,-6.0,2238.0,2898.0,15.38,Increasing
2,Arizona,513.0,9405.0,8708.0,5.36,Increasing
3,Arkansas,-25.0,9552.0,11105.0,15.63,Increasing
4,California,32.0,62574.0,80100.0,8.64,Decreasing


In [31]:
in_care_victims_perps_tpr_adopted = adopted.merge(in_care_victims_perps_tpr, how = "inner", on ="State")
in_care_victims_perps_tpr_adopted.head()

,State,Adopted Difference,TPR Difference,Unique count Perpetrators,Victim Population,"Victim Rate per 1,000",Status
0,Alabama,132.0,-234.0,7260.0,8601.0,7.63,Increasing
1,Alaska,60.0,-6.0,2238.0,2898.0,15.38,Increasing
2,Arizona,1328.0,513.0,9405.0,8708.0,5.36,Increasing
3,Arkansas,387.0,-25.0,9552.0,11105.0,15.63,Increasing
4,California,522.0,32.0,62574.0,80100.0,8.64,Decreasing
